In [1]:
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Load raw data from Google Drive
raw_data_path = "/content/drive/My Drive/dvc_storage/processed_sms_spam.csv"
data = pd.read_csv(raw_data_path)


Mounted at /content/drive


Install DVC and Google Support

In [3]:
!pip install dvc dvc[gdrive] -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.7/457.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.2/77.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.0/426.0 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.3/201.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.4 MB/s eta 0:00:00
   ━━━

In [10]:
import os
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
import dvc.api

# Download NLTK stopwords
nltk.download('stopwords')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Define dataset path in Google Drive
raw_data_path = "/content/drive/My Drive/dvc_storage/processed_sms_spam.csv"
data = pd.read_csv(raw_data_path)

# Convert category labels to binary (spam=1, ham=0)
data["target"] = data["category"].map({"spam": 1, "ham": 0})

# Preprocessing function
def preprocess_text(text):
    text = text.lower()
    regex = r"^subject:\s(.*)"
    match = re.search(regex, text)
    if match:
        text = match.group(1)
    text = re.sub(r"[^a-z .]", "", text)
    words = text.split()
    words = [word for word in words if word.isalpha() and word not in stopwords.words('english')]
    return ' '.join(words)

print("Begin text preprocessing:", end="\n\n")
data["processed_text"] = data["message"].apply(preprocess_text)
print("Preprocessing complete")

# First version (seed=42)
train_temp, test = train_test_split(data, test_size=0.2, random_state=42, stratify=data["target"])
train, validation = train_test_split(train_temp, test_size=0.3, random_state=42, stratify=train_temp["target"])

# Save splits
train.to_csv("train.csv", index=False)
validation.to_csv("validation.csv", index=False)
test.to_csv("test.csv", index=False)

# Initialize and track with DVC
os.system("dvc init --subdir")
os.system("dvc remote add -d storage gdrive://1dkzdzTtGUES5kMWg9lk5N6ECtmy2a5u3")
os.system("dvc add raw_data.csv train.csv validation.csv test.csv")
os.system("git add .")
os.system("git commit -m 'Initial data split with seed=42'")
os.system("dvc push")

# Updated split (seed=100)
train_temp_new, test_new = train_test_split(data, test_size=0.2, random_state=100, stratify=data["target"])
train_new, validation_new = train_test_split(train_temp_new, test_size=0.3, random_state=100, stratify=train_temp_new["target"])

# Save new splits
train_new.to_csv("train.csv", index=False)
validation_new.to_csv("validation.csv", index=False)
test_new.to_csv("test.csv", index=False)

# Update DVC
os.system("dvc add train.csv validation.csv test.csv")
os.system("git add .")
os.system("git commit -m 'Updated data split with seed=100'")
os.system("dvc push")

# Checkout previous version and print target distribution
os.system("git checkout HEAD~1")
os.system("dvc checkout")
for split, file in zip(["Train", "Validation", "Test"], ["train.csv", "validation.csv", "test.csv"]):
    df = pd.read_csv(file)
    print(f"{split} distribution:")
    print(df["target"].value_counts(), "\n")

# Checkout latest version
os.system("git checkout main")
os.system("dvc checkout")
for split, file in zip(["Train", "Validation", "Test"], ["train.csv", "validation.csv", "test.csv"]):
    df = pd.read_csv(file)
    print(f"{split} distribution:")
    print(df["target"].value_counts(), "\n")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Begin text preprocessing:

Preprocessing complete
Train distribution:
target
0    2701
1     418
Name: count, dtype: int64 

Validation distribution:
target
0    1158
1     180
Name: count, dtype: int64 

Test distribution:
target
0    966
1    149
Name: count, dtype: int64 

Train distribution:
target
0    2701
1     418
Name: count, dtype: int64 

Validation distribution:
target
0    1158
1     180
Name: count, dtype: int64 

Test distribution:
target
0    966
1    149
Name: count, dtype: int64 



Saving the CSVs to Google Drive

In [12]:
# Define dataset path in Google Drive
dvc_storage_path = "/content/drive/My Drive/dvc_storage/"
os.makedirs(dvc_storage_path, exist_ok=True)

# Save splits to Google Drive
train.to_csv(os.path.join(dvc_storage_path, "train.csv"), index=False)
validation.to_csv(os.path.join(dvc_storage_path, "validation.csv"), index=False)
test.to_csv(os.path.join(dvc_storage_path, "test.csv"), index=False)

print("Train, validation, and test sets saved in Google Drive at:", dvc_storage_path)


Train, validation, and test sets saved in Google Drive at: /content/drive/My Drive/dvc_storage/
